In [1]:
from functools import partial
import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from PIL import Image

In [27]:
#: plotting functions
dashed_line = partial(plt.plot, color='black', linestyle='--', linewidth=3)
highlight_line = partial(plt.plot, color='black', linestyle='-', linewidth=9)
text_plot = partial(plt.text, fontweight=100, fontsize=30)
vert_text = partial(text_plot, rotation=90, horizontalalignment='right', verticalalignment='center')
horz_text = partial(text_plot, horizontalalignment='center', verticalalignment='bottom')

def square_text(xy, s, **kwargs):
    x, y = xy
    text_plot(x + 0.5, y + 0.5, s, horizontalalignment='center', verticalalignment='center', **kwargs)
    return None
    

def square_plot(xy, val, color, **kwargs):
    square = plt.Rectangle(xy, facecolor=color, 
                           width=1, height=1, edgecolor='black', linewidth=3, **kwargs)
    plt.gca().add_patch(square)
    square_text(xy, str(val))
    return None
    

def highlight_plot(xy, color, alpha, **kwargs):
    square = plt.Rectangle(xy, facecolor=color, alpha=alpha, 
                           width=1, height=1, edgecolor='black', linewidth=9, **kwargs)
    plt.gca().add_patch(square)
    return None


def dashed_grid(width, height):
    for i in range(height + 1):
        dashed_line([0, width], [i, i])
    for j in range(width + 1):
        dashed_line([j, j], [0, height])
    return None

        
def fill_coo_array(coo, color):
    h, w = coo.shape
    for x, y, val in zip(coo.col, coo.row, coo.data):
        x, y = xy = (x, h - y - 1)
        rect = rect_plt(xy, facecolor=color)
        plt.gca().add_patch(rect)
        square_text(xy, str(val))
    return None
        

def format_graph(sparse_mtrx, fontsize):
    h, w = sparse_mtrx.shape
    title = sparse_mtrx.format.upper()
    text_plot((w + 1), (h + 0.1), title, verticalalignment='bottom', fontsize=(fontsize * 3))
    plt.text(0, -0.5, '© Matt Eding')
    plt.axis('equal')
    plt.axis('off')
    return None


def save_step(filename, step):
    plt.savefig(f'{filename}_{step:0>5}.png')
    plt.close()
    return None


def make_gif(filename, duration, start_extra=0, end_extra=0):
    cwd = Path.cwd()
    paths = sorted(p for p in cwd.iterdir() if 'png' in p.suffix)

    imgs = [Image.open(p) for p in paths]
    for _ in range(start_extra):
        imgs.insert(0, imgs[0])
        
    for _ in range(end_extra):
        imgs.append(imgs[-1])
        
    imgs[0].save(f'{filename}.gif', save_all=True, append_images=imgs, duration=duration, loop=0)
    
    for p in paths:
        p.unlink()
    
    return None

In [30]:
def coo_plot(coo, darkcolor, lightcolor, duration, filename='coo', fontsize=30, figsize=(20, 10)):
    plt.figure(figsize=figsize)
    h, w = coo.shape
    
    def plot_static_img():
        plt.figure(figsize=figsize)
        dashed_grid(w, h)
        fill_coo_array(coo, lightcolor)

        #: fill sparse format
        for i, (label, info) in enumerate(zip(['Row', 'Column', 'Data'], [coo.row, coo.col, coo.data])):
            text_plot((w + 1), (h - 0.75 - 2 * i), label)
            for j, val in enumerate(info):
                xy = ((w + j + 1), (h - 2 * (i + 1)))
                square_plot(xy, val, lightcolor)

        #: side labels
        vert_text(-1, (h / 2), 'Row')
        for i in reversed(range(h)):
            vert_text(0, (h - i - 0.5), str(i))

        horz_text((w / 2), (h + 1), 'Column')
        for i in range(w):
            horz_text(i + 0.5, h, str(i))
            
        format_graph(coo, fontsize)

    #: animate highlights
    step_counter = itertools.count()
    for i, (r, c) in enumerate(zip(row, col)):
        for a, alpha in enumerate(np.r_[0:1:10j, [1] * 10, 1:0:10j]):
            plot_static_img()
            for j in range(3):
                xy = ((w + i + 1), (h - 2 * (j + 1)))
                highlight_plot(xy, darkcolor, alpha)  # sparse part

            xy = (c, (h - r - 1))
            highlight_plot(xy, darkcolor, alpha)  # array part
            
            highlight_line([0, c], [h - r - 0.5] * 2, alpha=alpha)  # horz line
            highlight_line([c + 0.5] * 2, [h - r, h], alpha=alpha)  # vert line
            
            save_step(filename, next(step_counter))
    
    make_gif(filename, duration, start_extra=10)

In [38]:
row = [1, 3, 0, 2, 4]
col = [1, 4, 2, 3, 3]
data = [2, 5, 9, 1, 6]
coo = sparse.coo_matrix((data, (row, col)), shape=(6, 7))
coo_plot(coo, darkcolor='orchid', lightcolor='plum', duration=100)

<Figure size 1440x720 with 0 Axes>

In [35]:
def dia_plot(dia, darkcolor, lightcolor, duration, filename='dia', fontsize=30, figsize=(20, 10)):
    plt.figure(figsize=figsize)
    h, w = dia.shape
    
    def plot_static_img():
        plt.figure(figsize=figsize)
        dashed_grid(w, h)
        fill_coo_array(dia.tocoo(), lightcolor)

        #: fill sparse format
        text_plt((w + 1), (h - 0.75), 'Offsets')  # offsets
        for i, (val) in enumerate(dia.offsets):
            xy = ((w + i + 1), (h - 2))
            square_plot(xy, val, lightcolor)
        
        text_plt((w + 1), (h - 2.75), 'Data')  # data
        for i, row in enumerate(dia.data):
            for j, val in enumerate(row):
                xy = ((w + j + 1), (h - i - 4))
                square_plot(xy, val, lightcolor)
        
        #: side labels
        vert_text(-1, (h / 2), '− ⟵ Offset')  # unicode minus sign
        horz_text((w / 2), (h + 1), 'Offset ⟶ +')
        
        format_graph(dia, fontsize)
        
    
    #: animate highlights
    dur = 10
    step_counter = itertools.count()
    data = dia.tocoo().todia().data  # reformat dia.data to have falsy values where excess data
    lex = np.lexsort(dia.offsets[np.newaxis])  # new data did not preserve order
    for i, (offset, d_row) in enumerate(zip(dia.offsets, data[lex])):
        signed_zero = 0.0 * np.sign(offset)
        amount = abs(offset) + 1
        linspace = np.linspace(signed_zero, offset, amount)
        offs = np.r_[[np.nan] * dur, np.repeat(linspace, dur / 2), [offset] * 2 * dur]
        alphas = np.r_[[1] * (len(offs) - dur), 1:0:complex(dur)]

        for off, alpha in zip(offs, alphas):
            plot_static_img()
            
            #: unhighlighted
            if np.isnan(off):
                save_step(filename, next(step_counter))
                continue
            
            #: highlight offset
            xy = ((w + i + 1), (h - 2))
            highlight_plot(xy, darkcolor, alpha)

            #: highlight data
            for j, val in enumerate(d_row):
                xy = ((w + j + 1), (h - i - 4))
                color = darkcolor if val else 'none'
                highlight_plot(xy, color, alpha)

            #: highlight diagonals
            coo = sparse.eye(*dia.shape, k=int(off), dtype=dia.dtype).tocoo()
            for x, y, val in zip(coo.col, coo.row, coo.data):
                xy = (x, (h - y - 1))
                color = darkcolor if (off == offset) else 'none'
                highlight_plot(xy, color, alpha)
                
            #: side labeling for offset
            if offset <= 0:
                xy = (-0.6, (h + off))  # label on side
            else:
                xy = ((off - 0.6), h)  # label on top
            s = f'{off:+0.0f}'
            square_text(xy, s, alpha=alpha, fontweight=1000, 
                        ha='right', va='top')  # using ha/va to avoid functools.partial conflicts
        
            save_step(filename, next(step_counter))

    make_gif(filename, duration, start_extra=8)

In [36]:
data = np.arange(12).reshape(3, 4) + 1
offsets = np.array([0, -3, 2])
dia = sparse.dia_matrix((data, offsets), shape=(6, 4))
dia_plot(dia, 'darkkhaki', 'khaki', duration=100)

<Figure size 1440x720 with 0 Axes>